hyperparameter들을 바꿔가면서 괜찮은 모델을 만들어낸다.
- 기본 모델
    - one layer LSTM (+stateful)
    - batch size = 1
    - minmax 후 학습 -> 이후 다시 inverse_transform
    - (중요) unseen data에 대한 고려
    - train-test split - 7:3
- evaluation metric
    - MSE
- hyperparameters
    - `units` = 4
    - `epochs` = 100
    - `look_back` = 7

In [6]:
# LSTM for international airline passengers problem with memory
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
tf.random.set_seed(7)

In [21]:
# load the dataset
dataframe = read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

(100, 44)

In [22]:
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

(96, 40)

In [16]:
# create and fit the LSTM network
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(100):
	model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()

96/96 - 4s - loss: 0.0055 - 4s/epoch - 39ms/step
96/96 - 0s - loss: 0.0097 - 274ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0075 - 253ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0061 - 256ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0054 - 270ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0050 - 265ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0048 - 250ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0047 - 259ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0046 - 262ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0045 - 275ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0044 - 300ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0044 - 285ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0043 - 270ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0042 - 296ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0042 - 269ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0041 - 252ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0041 - 283ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0041 - 277ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0040 - 272ms/epoch - 3ms/step
96/96 - 0s - loss: 0.0040 - 257ms

In [19]:
# make predictions
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()

96/96 [==============================] - 0s 2ms/step


In [36]:
np.expand_dims(testX[0], axis=0).shape

(1, 3, 1)

In [37]:
# trainX.shape, testX.shape
model.predict(np.expand_dims(testX[0], axis=0).shape, batch_size=batch_size)

ValueError: in user code:

    File "c:\Users\govin\anaconda3\envs\aiffelthon\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\govin\anaconda3\envs\aiffelthon\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\govin\anaconda3\envs\aiffelthon\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\govin\anaconda3\envs\aiffelthon\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\govin\anaconda3\envs\aiffelthon\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\govin\anaconda3\envs\aiffelthon\lib\site-packages\keras\engine\input_

In [ ]:
testPredict = model.predict(testX, batch_size=batch_size)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()